In [ ]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 117.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#Instalações e imports
!pip install nltk pandas
import re, os
import pandas as pd
import nltk
import spacy
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt_tab')
nlp = spacy.load("pt_core_news_sm")

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.model_selection import StratifiedShuffleSplit

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:

#pre-processamento adaptado da atividade 2.
def normalizaEmails(texto):
    return re.sub(r'\b[\w\.-]+@[\w\.-]+(?:\.\w+)?\b[;,.]?', '<EMAIL>', texto)

def removeTagsHTML(texto):
    return re.sub(r'<[^>]+>', '', texto)

def normalizaNumeros(texto):
    return re.sub(r'\d+([.,]\d+)?', '<NUM>', texto)

def removeEspacosExtras(texto):
    return re.sub(r'\s+', ' ', texto).strip()

#tambem atrapalha a analise de sentimentos se não adptar a função.
def removeStopwordsNLTK(texto: str) -> str:
    """
    Remove stopwords do texto utilizando a lista de stopwords do NLTK para o português,
    MAS preserva sempre os negadores e os intensificadores.
    """
    negadores = {'não','nem','nunca','jamais','nao'}
    intensificadores = {'muito', 'super', 'extremamente', 'bem', 'tão', 'totalmente', 'mega', 'bastante','tao','super'}
    sw = set(stopwords.words('portuguese'))
    toks = nltk.word_tokenize(texto, language='portuguese')

    resultado = []
    for t in toks:
        tl = t.lower()
        # se for negação ou intensificador, mantém
        if tl in negadores or tl in intensificadores:
            resultado.append(t)
        # senão, apenas mantém se NÃO for stopword
        elif tl not in sw:
            resultado.append(t)
    return ' '.join(resultado)

#na analise de sentimentos realizaStemmingNLTK , atrapalha, lexico não trabalha com radicais.
#não usaremos, mas dexarei aqui para recordar
def realizaStemmingNLTK(texto):
    stemmer = RSLPStemmer()
    toks = nltk.word_tokenize(texto, language='portuguese')
    return ' '.join([stemmer.stem(t) for t in toks])



def preprocessamentoDeTextoNLTK(texto: str) -> str:
    texto = normalizaEmails(texto)
    texto = removeTagsHTML(texto)
    texto = normalizaNumeros(texto)
    texto = removeEspacosExtras(texto)
    texto = removeStopwordsNLTK(texto)
    return texto



In [ ]:
def carrega_lacafe(path_lexico: str):
    """
    Lê o arquivo AffectPT‑br (Lacafe) e retorna dois conjuntos de stems:
      LEX_POS_STEMS, LEX_NEG_STEMS.
    """
    LEX_POS_STEMS = set()
    LEX_NEG_STEMS = set()
    with open(path_lexico, encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('%'):
                continue
            parts = re.split(r'\s+', line)
            # parts = [palavra, código_gram, código_cat1, código_cat2, ...]
            palavra = parts[0]
            cats = set(parts[2:])  # ex: {'31','32',...}
            if '31' in cats:       # posemo → positivo
                LEX_POS_STEMS.add(palavra.rstrip('*'))
            if '32' in cats:       # negemo → negativo
                LEX_NEG_STEMS.add(palavra.rstrip('*'))
    return LEX_POS_STEMS, LEX_NEG_STEMS


LEX_POS_STEMS, LEX_NEG_STEMS = carrega_lacafe('AffectPT-br.txt')
#le o CSV de treino
colunas = [
    'object',
    'text',
    'stars',
    'discrete_stars'
]

FILENAME = 'test_movies.csv'

#Carrega o CSV forçando header=None e usando 'names=colunas', pulando a 1ª linha
df = pd.read_csv(
    FILENAME,
    sep=',',
    quotechar='"',       # importante para respeitar o campo text que pode ter vírgulas
    engine='python',     # parser Python lida melhor com strings mal formatadas
    on_bad_lines='skip', # ignora linhas que ainda destoem do padrão
    header=None,         # diz ao Pandas “não procure header no arquivo”
    names=colunas,       # e aqui é o único header
    skiprows=1,          # pula a PRIMEIRA linha do arquivo (que esta corrompida)
    encoding='utf-8'
)

def score_sentiment_com_negacao(texto: str) -> int:
    """
    Score = +1 para cada token que corresponde a um stem positivo,
            −1 para cada stem negativo,
            +2 ou −2 se houver intensificador antes,
            com inversão se precedido de negação.
    """
    tokens = nltk.word_tokenize(texto, language='portuguese')
    score = 0
    negadores = {'não','nem','nunca','jamais','nao'}
    intensificadores = {'muito', 'super', 'extremamente', 'bem', 'tão', 'totalmente', 'mega', 'bastante','tao','Super'}

    i = 0
    #para poder usar intensificadores
    while i < len(tokens):
        tl = tokens[i].lower()

        # Se for negação e tiver uma palavra depois
        if tl in negadores and i+1 < len(tokens):
            nxt = tokens[i+1].lower()
            if any(nxt.startswith(stem) for stem in LEX_POS_STEMS):
                score -= 1
                i += 2
                continue
            if any(nxt.startswith(stem) for stem in LEX_NEG_STEMS):
                score += 1
                i += 2
                continue

        # Se for intensificador e tiver palavras em janela de uma palavra
        if tl in intensificadores and i+1 < len(tokens):
            nxt = tokens[i+1].lower()
            if any(nxt.startswith(stem) for stem in LEX_POS_STEMS):
                score += 2
                i += 2
                continue
            if any(nxt.startswith(stem) for stem in LEX_NEG_STEMS):
                score -= 2
                i += 2
                continue

        # Caso normal
        if any(tl.startswith(stem) for stem in LEX_POS_STEMS):
            score += 1
        if any(tl.startswith(stem) for stem in LEX_NEG_STEMS):
            score -= 1

        i += 1

    return score

df['text_pp'] = df['text'].fillna('').astype(str).apply(preprocessamentoDeTextoNLTK)
df['lex_score'] = df['text_pp'].apply(score_sentiment_com_negacao)

# calcula percentis
def map_score_faixa(s):
    if s <= -1: return 1
    elif s <= 0: return 2
    elif s <= 1: return 3
    elif s <= 2: return 4
    else:        return 5

df['lex_stars_q'] = df['lex_score'].apply(map_score_faixa)

# ========================

print("Distribuição léxica (percentis):")
print(df['lex_stars_q'].value_counts().sort_index())


print("Distribuição real de stars:")
print(df['discrete_stars'].value_counts().sort_index())

print("diferença")
print(df['lex_stars_q'].value_counts().sort_index() - df['discrete_stars'].value_counts().sort_index())

# sorteia 20 exemplos para avaliação qualitativa
amostra_qualitativa = df.sample(n=20, random_state=28)[
    ['text_pp', 'discrete_stars', 'lex_score', 'lex_stars_q']
]

print("=== Amostra Qualitativa (20 reviews aleatórias) ===")
print(amostra_qualitativa.to_string(index=False))


Distribuição léxica (percentis):
lex_stars_q
1    1719
2    2741
3    2139
4    1270
5    1543
Name: count, dtype: int64
Distribuição real de stars:
discrete_stars
1    1929
2    1818
3    1865
4    1839
5    1961
Name: count, dtype: int64
diferença
lex_stars_q
1   -210
2    923
3    274
4   -569
5   -418
Name: count, dtype: int64
=== Amostra Qualitativa (20 reviews aleatórias) ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support
)

# 1. Vetores verdade (y_true) e predição (y_pred)
y_true = df['discrete_stars'].astype(int)
y_pred = df['lex_stars_q'].astype(int)

# 2. Acurácia (só para referência)
acc = accuracy_score(y_true, y_pred)
print(f"Acurácia geral: {acc:.3f}")

# 3. Relatório completo: precisão, recall e F1 para cada classe,
#    + médias macro, micro e weighted
print("\nRelatório de classificação:")
print(classification_report(
        y_true,
        y_pred,
        digits=3,           # casas decimais
        zero_division=0     # evita Warning se faltar classe
))

# 4. Matriz de confusão
labels = [1, 2, 3, 4, 5]           # garante que apareçam na ordem certa
cm = confusion_matrix(y_true, y_pred, labels=labels)
print("Matriz de confusão (linhas = verdade, colunas = previsto):")
print(confusion_matrix(y_true, y_pred))

cm_df = pd.DataFrame(
    cm,
    index  =[f"Real ★{l}" for l in labels],      # linhas rotuladas
    columns=[f"Previsto ★{l}" for l in labels]   # colunas rotuladas
)
print("\nMatriz de Confusão (rotulada):")
print(cm_df)


Acurácia geral: 0.214

Relatório de classificação:
              precision    recall  f1-score   support

           1      0.201     0.165     0.181      1929
           2      0.218     0.336     0.264      1818
           3      0.191     0.246     0.215      1865
           4      0.230     0.154     0.184      1839
           5      0.250     0.176     0.207      1961

    accuracy                          0.214      9412
   macro avg      0.218     0.215     0.210      9412
weighted avg      0.218     0.214     0.210      9412

Matriz de confusão (linhas = verdade, colunas = previsto):
[[319 654 573 228 155]
 [506 611 347 177 177]
 [313 527 458 254 313]
 [220 481 460 283 395]
 [233 534 557 291 346]]

Matriz de Confusão (rotulada):
         Previsto ★1  Previsto ★2  Previsto ★3  Previsto ★4  Previsto ★5
Real ★1          319          654          573          228          155
Real ★2          506          611          347          177          177
Real ★3          313          527 